In [3]:
# zones_to_base_i2k_dual_safe.py
# 生成 i,k,base_minutes,dist_km；支持 OSRM 与 欧几里得两种方式，并采用分块流式写出避免内存爆。

from pathlib import Path
from typing import Iterator, Tuple
import math
import numpy as np
import pandas as pd

# ===================== 可配置常量 =====================
METHOD = "EUCLID"                 # 选 "EUCLID" 或 "OSRM"

ZONES_CSV    = r"../data/zones.csv"                 # 需包含列：zone,lat,lon
STATIONS_CSV = r"../data/stations.csv"     # 需包含列：k,lat,lon（其余列忽略）
OUT_CSV      = r"../data/base_i2k.csv"              # 输出 CSV（i,k,base_minutes,dist_km）

FILL_MINUTES_IF_NAN = None        # 仅 OSRM 有效：若时长为 NaN，用该值（分钟）回填；None 不回填

# —— 欧几里得模式（直线距离 + 平均速度）
SPEED_KMPH_EUCLID = 35.0          # km/h，用于把直线距离换算成分钟
ROW_BLOCK = 800                   # zones 分块大小（行块），越大越快但更吃内存
COL_BLOCK = 800                   # stations 分块大小（列块）

# —— OSRM 模式（道路距离/时间；需 osm_distance.py 可导入）
OSRM_BASE_URL = "http://router.project-osrm.org"
OSRM_PROFILE  = "driving"
OSRM_TABLE_BATCH = 80
OSRM_USE_TABLE = True
OSRM_CACHE_FILE = "data/cache/osm_distance_cache.pkl"
ORIG_BLOCK = 60                   # OSRM: zones 起点分块大小
DEST_BLOCK = 60                   # OSRM: stations 终点分块大小
# =====================================================


def _require_cols(df: pd.DataFrame, need: list[str]):
    miss = [c for c in need if c not in df.columns]
    if miss:
        raise ValueError(f"缺少列 {miss}；实际列：{list(df.columns)}")


def load_zones(zones_csv: str) -> pd.DataFrame:
    z = pd.read_csv(zones_csv, dtype={"zone": str})
    _require_cols(z, ["zone", "lat", "lon"])
    z["lat"] = pd.to_numeric(z["lat"], errors="coerce")
    z["lon"] = pd.to_numeric(z["lon"], errors="coerce")
    z = z.dropna(subset=["lat","lon"]).drop_duplicates(["zone"]).reset_index(drop=True)
    return z[["zone","lat","lon"]]


def load_stations(st_csv: str) -> pd.DataFrame:
    s = pd.read_csv(st_csv, dtype={"k": str})
    # 兼容大小写/别名
    cols = {c.lower(): c for c in s.columns}
    kcol  = cols.get("k") or cols.get("id") or cols.get("station_id")
    latc  = cols.get("lat") or cols.get("latitude")
    lonc  = cols.get("lon") or cols.get("lng") or cols.get("longitude")
    if not all([kcol, latc, lonc]):
        raise ValueError("stations CSV 需包含列 [k, lat, lon]（大小写或同义名均可）")
    s = s[[kcol, latc, lonc]].rename(columns={kcol:"k", latc:"lat", lonc:"lon"})
    s["lat"] = pd.to_numeric(s["lat"], errors="coerce")
    s["lon"] = pd.to_numeric(s["lon"], errors="coerce")
    s = s.dropna(subset=["lat","lon"]).drop_duplicates(["k"]).reset_index(drop=True)
    s["k"] = s["k"].astype(str)
    return s[["k","lat","lon"]]


def tile_ranges(n: int, r: int) -> Iterator[Tuple[int, int]]:
    """生成 [0,n) 的分块区间 [i0,i1)"""
    i = 0
    while i < n:
        j = min(n, i + r)
        yield i, j
        i = j


# ------------------ 欧几里得模式（分块流式） ------------------
def build_by_euclid_stream(z: pd.DataFrame, st: pd.DataFrame, out_csv: str):
    zid = z["zone"].to_numpy(str)     # (Nz,)
    zlat = z["lat"].to_numpy(float)
    zlon = z["lon"].to_numpy(float)

    kid = st["k"].to_numpy(str)       # (Nk,)
    klat = st["lat"].to_numpy(float)
    klon = st["lon"].to_numpy(float)

    Nz, Nk = len(z), len(st)

    R = 6371.0088  # 地球半径 km
    zlat_r = np.radians(zlat)
    zlon_r = np.radians(zlon)
    klat_r = np.radians(klat)
    klon_r = np.radians(klon)

    first_chunk = True
    Path(out_csv).parent.mkdir(parents=True, exist_ok=True)

    for i0, i1 in tile_ranges(Nz, ROW_BLOCK):
        # 行块（zones）
        lat_row = zlat_r[i0:i1][:, None]  # (R,1)
        lon_row = zlon_r[i0:i1][:, None]  # (R,1)
        ids_i   = zid[i0:i1]              # (R,)

        for j0, j1 in tile_ranges(Nk, COL_BLOCK):
            # 列块（stations）
            lat_col = klat_r[None, j0:j1] # (1,C)
            lon_col = klon_r[None, j0:j1] # (1,C)
            ids_k   = kid[j0:j1]          # (C,)

            # equirectangular 近似：直线距离
            dlat = lat_col - lat_row                  # (R,C)
            dlon = lon_col - lon_row                  # (R,C)
            x = dlon * np.cos((lat_row + lat_col)/2.) # (R,C)
            y = dlat
            dist_km = R * np.sqrt(x*x + y*y)          # (R,C)

            # 距离 -> 分钟
            base_minutes = (dist_km / max(1e-9, float(SPEED_KMPH_EUCLID))) * 60.0

            # 展平为列
            I = np.repeat(ids_i, j1-j0)              # (R*C,)
            K = np.tile(ids_k,  i1-i0)               # (R*C,)
            m = base_minutes.ravel()
            d = dist_km.ravel()

            df_chunk = pd.DataFrame({
                "i": I,
                "k": K,
                "base_minutes": m.astype(float),
                "dist_km": d.astype(float),
            })

            df_chunk.to_csv(out_csv, index=False, mode="w" if first_chunk else "a",
                            header=first_chunk)
            first_chunk = False

        print(f"[EUCLID] zones {i1}/{Nz} (~{i1/Nz:.1%}) 已写出...")

    print(f"[EUCLID] 完成：{out_csv}")


# ------------------ OSRM 模式（分块流式） ------------------
def build_by_osrm_stream(z: pd.DataFrame, st: pd.DataFrame, out_csv: str):
    try:
        from osm_distance import OSRMDistanceCalculator, OSRMConfig
    except Exception as e:
        raise ImportError("无法导入 osm_distance.py，请将其放在同目录或 PYTHONPATH 中。") from e

    cfg = OSRMConfig(
        base_url=OSRM_BASE_URL,
        profile=OSRM_PROFILE,
        table_batch=OSRM_TABLE_BATCH,
        use_table_api=OSRM_USE_TABLE,
    )
    calc = OSRMDistanceCalculator(osrm_cfg=cfg, cache_file=OSRM_CACHE_FILE)

    O = z.rename(columns={"zone": "id"})[["id","lat","lon"]].copy()
    D = st.rename(columns={"k": "id"})[["id","lat","lon"]].copy()
    Nz, Nk = len(O), len(D)

    first_chunk = True
    Path(out_csv).parent.mkdir(parents=True, exist_ok=True)

    for i0, i1 in tile_ranges(Nz, ORIG_BLOCK):
        O_blk = O.iloc[i0:i1].copy()
        for j0, j1 in tile_ranges(Nk, DEST_BLOCK):
            D_blk = D.iloc[j0:j1].copy()

            df = calc.matrix(
                origins=O_blk, destinations=D_blk,
                origin_id_col="id", dest_id_col="id",
                lat_col="lat", lon_col="lon"
            )
            # df: origin_id, destination_id, distance_km, duration_min, source
            df = df.rename(columns={
                "origin_id": "i", "destination_id": "k",
                "duration_min": "base_minutes", "distance_km": "dist_km"
            })[["i","k","base_minutes","dist_km"]]

            if FILL_MINUTES_IF_NAN is not None:
                df["base_minutes"] = pd.to_numeric(df["base_minutes"], errors="coerce").fillna(float(FILL_MINUTES_IF_NAN))

            df.to_csv(out_csv, index=False, mode="w" if first_chunk else "a",
                      header=first_chunk)
            first_chunk = False

        print(f"[OSRM] zones {i1}/{Nz} (~{i1/Nz:.1%}) 已写出...")

    print(f"[OSRM] 完成：{out_csv}")


def main():
    zones = load_zones(ZONES_CSV)
    stations = load_stations(STATIONS_CSV)

    if METHOD.upper() == "EUCLID":
        build_by_euclid_stream(zones, stations, OUT_CSV)
    elif METHOD.upper() == "OSRM":
        build_by_osrm_stream(zones, stations, OUT_CSV)
    else:
        raise ValueError("METHOD 必须为 'EUCLID' 或 'OSRM'")

    # 简单检查：读取前几行
    head = pd.read_csv(OUT_CSV, nrows=5)
    print(head)


if __name__ == "__main__":
    main()


[EUCLID] zones 221/221 (~100.0%) 已写出...
[EUCLID] 完成：../data/base_i2k.csv
     i  k  base_minutes   dist_km
0  308  1      8.595977  5.014320
1  308  2      8.396878  4.898179
2  308  3      8.396297  4.897840
3  308  4      8.393829  4.896400
4  308  5      8.391142  4.894833


In [4]:
import pandas as pd

# 读 CSV
df = pd.read_csv("../data/base_i2k.csv")  # 可能需要指定 encoding、sep、dtype 等

# 写 Parquet（需要安装 pyarrow 或 fastparquet）
df.to_parquet("../data/base_i2k.parquet", index=False)